# Init

In [1]:
import os
import math

import numpy as np
import fasttext.util

import nn
import utils

%load_ext autoreload
%autoreload 2

MODEL_DIM = 64
INNER_DIM = MODEL_DIM * 4

SEQUENCE_MAX = 128 # Max length of input or output sentence
NUM_HEADS = 8

In [2]:
embeddings = utils.get_embeddings(["en", "fr"], dim=MODEL_DIM) # using dim 256 instead of 512

en_emb = embeddings["en"]
fr_emb = embeddings["fr"]

print(en_emb.get_dimension())
print(fr_emb.get_dimension())

64
64


In [3]:
def pad_embedding(x, pad_len, embed_dim):
    """Pads a word embedding to a max length"""
    padded = np.zeros((pad_len - x.shape[0], embed_dim))
    return np.concatenate((x, padded))


class TranslationDataset():
    """Dataset for the position encoded and word embedded translations"""
    def __init__(self, inputs, targets, embeddings, pad_len, embed_dim):
        self.inputs = inputs 
        self.targets = targets 
        
        # Encoders for both languages
        emb_in = embeddings["en"]
        emb_tgt = embeddings["fr"]
        
        # Embed all inputs
        self.input_em = []
        for seq in inputs:
            embed = np.array([emb_in.get_word_vector(w) for w in seq.split()])
            embed = pad_embedding(embed, pad_len, embed_dim)
            self.input_em.append(np.array(embed))
            
        # Embed all outputs 
        self.target_em = []
        for seq in targets:
            embed = np.array([emb_tgt.get_word_vector(w) for w in seq.split()])
            embed = pad_embedding(embed, pad_len, embed_dim)
            self.target_em.append(np.array(embed))
    
    def pad_embedding(self, x):
        padded = np.zeros((self.pad_length - x.shape[0], self.embed_dim))
        return np.concatenate((x, padded))

    def __len__(self):
        return (len(self.sequence))

    def __getitem__(self, idx):
        return({
            "input":self.inputs[idx],
            "target":self.targets[idx],
            "input_embedding":self.input_em[idx],
            "target_embedding":self.target_em[idx],
        })
    
data = ["the cat likes oranges", "hello my friend"]
target = ["le chat aime les oranges", "bonjour mon amie"]

dataset = TranslationDataset(data, target, embeddings, SEQUENCE_MAX, MODEL_DIM) 

In [4]:
def pos_encoding(seq):
    """Adds positional encoding to a sequence of word vectors"""
    seq_len = seq.shape[0]
    d_model = seq.shape[1]
    
    encoding = []
    for i in range(seq_len):
        w = 1 / (10000 ** ((2 * i) / d_model))

        wi_s = [math.sin(p * w) * (i % 2) for p in range(d_model)]
        wi_c = [math.cos(p * w) * ((i + 1) % 2) for p in range(d_model)]
        
        encoding.append(np.add(wi_s, wi_c))
        
    encoding = np.array(encoding)
    
    return np.add(encoding, seq)

In [43]:
class AttentionHead():
    """Scaled dot product attention head. """
    def __init__(self, embed_dim, n_heads):
        self.embed_dim = embed_dim
        self.n_heads = n_heads 
        self.output_dim = embed_dim // n_heads
        
        self.layers = [nn.LinearLayer(embed_dim, self.output_dim)
                       for _ in range(3)]
        
        #TODO make cleaner
        self.V, self.K, self.Q = self.layers[0], self.layers[1], self.layers[2]
    
    def __call__(self, x, mask=None):
        """Attention forward pass"""
        d_k = self.embed_dim // self.output_dim
        scale = math.sqrt(d_k)
        
        scaled_dp = np.dot(self.Q(x.T), self.K(x.T).T) / scale
        
        if mask is not None:
            scaled_dp = np.add(mask, scaled_dp)
            
        return(np.matmul(nn.softmax(scaled_dp), self.V(x.T)))
        
        
class MultiheadAttention():
    """Multiheaded attention transformer block"""
    def __init__(self, embed_dim=MODEL_DIM, n_heads=NUM_HEADS, masked=False):
        # Class vars
        self.n_heads = n_heads
        self.embed_dim = embed_dim
        self.step_len = self.embed_dim // self.n_heads
        
        # Create all heads and weights for multiheaded attention
        self.heads = [AttentionHead(embed_dim, n_heads) for _ in range(n_heads)]
        self.O = nn.LinearLayer(embed_dim, embed_dim)
        
        self.attn_dim = embed_dim // n_heads
        
        # If the attention block is masked
        self.mask = None
        if masked:
            self.mask = np.ones((self.attn_dim, self.attn_dim)) * -np.inf
            self.mask = np.triu(self.mask, k=1)
    
    def __call__(self, x):
        """Forward passes through concatenated heads and matmuls by weights"""
        h_cat = np.concatenate(np.array([h(x, self.mask) for h in self.heads]))
        return(self.O(h_cat))
        
    def backprop(self, x, t, alpha=0.001):
        # Makes one pass across all concatonated heads of multiheaded attention
        for i, h in enumerate(self.heads):
            t = x[0 + i * self.step_len, (1 + i) * self.step_len]
            for layer in h.layers:
                layer.weights -= layer.delta(x, t).T * alpha
        

In [6]:
class FFN(nn.Net):
    """Position-wise feed forward nueral network"""
    def __init__(self, embed_dim=MODEL_DIM, inner_dim=INNER_DIM, sequence_max=SEQUENCE_MAX):
        super(FFN, self).__init__()
        self.L1 = nn.LinearLayer(embed_dim, inner_dim, sequence_max)
        self.L2 = nn.LinearLayer(inner_dim, embed_dim, sequence_max)
        
        self.layers = [
            self.L1,
            self.L2,
        ]
    
    def __call__(self, x):
        """ReLU(xW1 + b1)W2 + b2"""
        x = self.L2(np.maximum(self.L1(x), 0))
        
        return x

In [7]:
#TODO implement layer norm
def layer_norm(layer):
    std = np.std(layer)
    mean = np.mean(layer)
    print(std, mean)
    print(layer)


In [44]:
class EncoderBlock():
    
    """Encoder block for the transformer.
    Args:
            embed_dim (string): Directory with all the images.
            n_heads (string): Path to the csv file with annotations.
            sequence_max (int):
            inner_dim (int):
    """
    def __init__(self, embed_dim=MODEL_DIM, n_heads=NUM_HEADS, 
                 inner_dim=INNER_DIM, sequence_max=SEQUENCE_MAX):
        self.embed_dim = embed_dim
        self.n_heads = n_heads 
        self.sequence_max = sequence_max
        
        self.multihead_attn = MultiheadAttention(embed_dim, n_heads)
        self.feedforward = FFN(embed_dim, inner_dim, sequence_max)
    
    def temp_add_norm(self, x, y):
        # TODO replace with layer norm function
        x = np.add(x, y) 
        return (x / np.sqrt(np.sum(x**2))).T
        
    def __call__(self, x):
        y = self.multihead_attn(x)
        x = self.temp_add_norm(x, y.T)
        
        y = self.feedforward(x)
        x = self.temp_add_norm(x, y)
        
        return x

multihead_attn = MultiheadAttention()
ffn = FFN()

x = dataset[0]["input_embedding"]
t = dataset[0]["input_embedding"]

x = multihead_attn(x)

multihead_attn.backprop(x, t)


#x = ffn(t.T)
#
#
#ffn.backprop(x, t)
#print(nn.loss(x, t))

#for i in range(x.shape[1]):
#    print(x[:,i].shape, t.T[:,i].shape)
#    ffn.backprop(x[:,i], t.T[:,i])
#    print(nn.loss(x, t.T))
    

#layer_norm(x)
#enc_block = EncoderBlock()